## 批量归一化 — 使用tensorflow
本章介绍如何使用tensorflow在训练和测试深度学习模型中使用批量归一化。

### 定义模型并添加批量归一化层
有了tensorflow，我们模型的定义工作变得简单了许多。我们只需要添加slim.batch_norm层并指定对二维卷积的通道(axis=1)进行批量归一化。



In [1]:
import tensorflow as tf
slim = tf.contrib.slim

def net(input, is_training):
    with tf.variable_scope('lenet'):
        conv1 = slim.conv2d(input, 20, [5, 5], scope='conv1_1', activation_fn=None,weights_initializer=tf.random_normal_initializer(stddev=0.01))
        conv1 = slim.batch_norm(conv1, decay=0.9, is_training=is_training, updates_collections=None)
        conv1 = tf.nn.relu(conv1)
        
        pool1 = slim.max_pool2d(conv1, [2, 2], scope='pool_1')
        
        conv2 = slim.conv2d(pool1, 50, [3, 3], scope='conv2_2', activation_fn=None, weights_initializer=tf.random_normal_initializer(stddev=0.01))
        conv1 = slim.batch_norm(conv1, decay=0.9, is_training=is_training, updates_collections=None)
        conv1 = tf.nn.relu(conv1)
        
        pool2 = slim.max_pool2d(conv2, [2, 2], scope='pool_2')
        pool2 = slim.flatten(pool2)
        fc1 = slim.fully_connected(pool2, 128, scope='fc1', activation_fn=None, weights_initializer=tf.random_normal_initializer(stddev=0.01))
        fc1 = tf.nn.relu(fc1)
        fc2 = slim.fully_connected(fc1, 10, scope='fc2', activation_fn=None, weights_initializer=tf.random_normal_initializer(stddev=0.01))
        return conv1, conv2, fc1, fc2

In [2]:
import sys
import numpy as np

sys.path.append('../../utils')
import utils

data_dir = '../../data/fashion_mnist'
train_images, train_labels, test_images, test_labels = utils.load_data_fashion_mnist(data_dir, one_hot=True)
print train_images.shape
print test_images.shape

from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
train_dataset = DataSet(train_images, train_labels, one_hot=True)
test_dataset = DataSet(test_images, test_labels, one_hot=True)

learning_rate = 1e-2
max_steps = 1000
batch_size = 256
height = width = 28
num_channels = 1
num_outputs = 10

#tf.reset_default_graph()

input_placeholder = tf.placeholder(tf.float32, [None, height, width, num_channels])
#input_placeholder = tf.placeholder(tf.float32, [None, height*width*num_channels])
is_training = tf.placeholder(tf.bool)
gt_placeholder = tf.placeholder(tf.int64, [None, num_outputs])
conv1, conv2, fc1, logits = net(input_placeholder, is_training)
loss = tf.losses.softmax_cross_entropy(logits=logits,  onehot_labels=gt_placeholder)
acc = utils.accuracy(logits, gt_placeholder)
test_images_reshape = np.expand_dims(np.reshape(np.squeeze(test_images), (test_images.shape[0], height, width)), axis=-1)
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
test_acc = []
test_loss = []

Extracting ../../data/fashion_mnist/train-images-idx3-ubyte.gz
Extracting ../../data/fashion_mnist/train-labels-idx1-ubyte.gz
Extracting ../../data/fashion_mnist/t10k-images-idx3-ubyte.gz
Extracting ../../data/fashion_mnist/t10k-labels-idx1-ubyte.gz
(60000, 28, 28, 1)
(10000, 28, 28, 1)
> <ipython-input-1-0737824316c7>(8)net()
-> conv1 = slim.conv2d(input, 20, [5, 5], scope='conv1_1', activation_fn=None,weights_initializer=tf.random_normal_initializer(stddev=0.01))
(Pdb) c
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [3]:
for step in range(max_steps):
    data, label = train_dataset.next_batch(batch_size)
    data = np.reshape(data, (batch_size, height, width, num_channels))
    feed_dict = {input_placeholder: data, gt_placeholder: label, is_training: True}
    loss_, acc_, _ = sess.run([loss, acc, train_op], feed_dict=feed_dict)    
    if step % 10 == 0:
        print("Batch %d, Loss: %f, Train acc %f " % (step, loss_, acc_))
        test_loss_, test_acc_ = sess.run([loss, acc], feed_dict={input_placeholder: test_images_reshape / 255.0, gt_placeholder: test_labels, is_training: False})

        print ("Test Loss: %f, Test acc %f " % (test_loss_, test_acc_))

#print ("Test acc %f " % (np.mean(test_acc)))

Batch 0, Loss: 2.302401, Train acc 0.066406 
Test Loss: 2.302563, Test acc 0.100000 
Batch 10, Loss: 2.300360, Train acc 0.109375 
Test Loss: 2.302372, Test acc 0.100000 
Batch 20, Loss: 2.294809, Train acc 0.273438 
Test Loss: 2.301208, Test acc 0.100200 
Batch 30, Loss: 2.264817, Train acc 0.378906 
Test Loss: 2.286034, Test acc 0.332900 
Batch 40, Loss: 1.804419, Train acc 0.457031 
Test Loss: 1.751478, Test acc 0.391500 
Batch 50, Loss: 1.460266, Train acc 0.429688 
Test Loss: 1.345761, Test acc 0.482500 
Batch 60, Loss: 1.199955, Train acc 0.574219 
Test Loss: 1.448195, Test acc 0.447100 
Batch 70, Loss: 1.264191, Train acc 0.449219 
Test Loss: 1.165174, Test acc 0.503500 
Batch 80, Loss: 1.101233, Train acc 0.542969 
Test Loss: 1.161045, Test acc 0.582100 
Batch 90, Loss: 0.876922, Train acc 0.632812 
Test Loss: 0.898799, Test acc 0.643500 
Batch 100, Loss: 0.941172, Train acc 0.601562 
Test Loss: 1.034490, Test acc 0.595900 
Batch 110, Loss: 0.735028, Train acc 0.730469 
Test Lo

Test Loss: 0.321026, Test acc 0.881400 
Batch 950, Loss: 0.228617, Train acc 0.910156 
Test Loss: 0.324394, Test acc 0.882200 
Batch 960, Loss: 0.263707, Train acc 0.910156 
Test Loss: 0.335900, Test acc 0.875700 
Batch 970, Loss: 0.293388, Train acc 0.886719 
Test Loss: 0.338327, Test acc 0.876600 
Batch 980, Loss: 0.245423, Train acc 0.906250 
Test Loss: 0.344116, Test acc 0.873300 
Batch 990, Loss: 0.308832, Train acc 0.894531 
Test Loss: 0.334100, Test acc 0.877500 


# 